In [1]:
import polars as pl
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from datetime import date
import config
from Preprocessing.ChartPatternAlgorithms import *
import utils
import importlib

output_notebook()

importlib.reload(utils)

Loading BokehJS ...

<module 'utils' from 'c:\\repo\\DeepTrader\\utils.py'>

In [2]:
btc = (
    pl.scan_csv(config.btc_data)
    .rename(
    {'open_time': 'time'}
    )
    .drop(
        ['close_time', 'quote_volume', 'taker_buy_volume', 'taker_buy_quote_volume', 'ignore', 'count']
    )
).collect()

print(btc)


shape: (38_880, 6)
┌───────────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ time          ┆ open    ┆ high    ┆ low     ┆ close   ┆ volume  │
│ ---           ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ i64           ┆ f64     ┆ f64     ┆ f64     ┆ f64     ┆ f64     │
╞═══════════════╪═════════╪═════════╪═════════╪═════════╪═════════╡
│ 1675209600000 ┆ 23119.5 ┆ 23134.2 ┆ 23119.4 ┆ 23129.9 ┆ 449.836 │
│ 1675209660000 ┆ 23130.0 ┆ 23131.2 ┆ 23117.4 ┆ 23117.6 ┆ 286.361 │
│ 1675209720000 ┆ 23117.6 ┆ 23135.8 ┆ 23117.6 ┆ 23135.6 ┆ 213.671 │
│ 1675209780000 ┆ 23135.5 ┆ 23161.4 ┆ 23135.5 ┆ 23154.1 ┆ 628.889 │
│ …             ┆ …       ┆ …       ┆ …       ┆ …       ┆ …       │
│ 1677628560000 ┆ 23105.9 ┆ 23108.0 ┆ 23103.8 ┆ 23106.8 ┆ 81.839  │
│ 1677628620000 ┆ 23106.8 ┆ 23124.6 ┆ 23106.2 ┆ 23107.1 ┆ 287.979 │
│ 1677628680000 ┆ 23107.0 ┆ 23114.7 ┆ 23107.0 ┆ 23110.1 ┆ 130.614 │
│ 1677628740000 ┆ 23110.0 ┆ 23130.0 ┆ 23110.0 ┆ 23129.6 ┆ 221.399 │
└───────────────┴─────────┴──

In [222]:
local_tops = rw_tops(data = btc['close'], order = 2)

tops = btc[::-1].with_columns(local_tops)

print(tops)

shape: (38_880, 7)
┌───────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────┐
│ time          ┆ open    ┆ high    ┆ low     ┆ close   ┆ volume  ┆ Tops  │
│ ---           ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---   │
│ i64           ┆ f64     ┆ f64     ┆ f64     ┆ f64     ┆ f64     ┆ bool  │
╞═══════════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═══════╡
│ 1677628740000 ┆ 23110.0 ┆ 23130.0 ┆ 23110.0 ┆ 23129.6 ┆ 221.399 ┆ false │
│ 1677628680000 ┆ 23107.0 ┆ 23114.7 ┆ 23107.0 ┆ 23110.1 ┆ 130.614 ┆ false │
│ 1677628620000 ┆ 23106.8 ┆ 23124.6 ┆ 23106.2 ┆ 23107.1 ┆ 287.979 ┆ false │
│ 1677628560000 ┆ 23105.9 ┆ 23108.0 ┆ 23103.8 ┆ 23106.8 ┆ 81.839  ┆ false │
│ …             ┆ …       ┆ …       ┆ …       ┆ …       ┆ …       ┆ …     │
│ 1675209780000 ┆ 23135.5 ┆ 23161.4 ┆ 23135.5 ┆ 23154.1 ┆ 628.889 ┆ false │
│ 1675209720000 ┆ 23117.6 ┆ 23135.8 ┆ 23117.6 ┆ 23135.6 ┆ 213.671 ┆ false │
│ 1675209660000 ┆ 23130.0 ┆ 23131.2 ┆ 23117.4 ┆ 23117.6 ┆ 286.361 ┆ f

In [3]:
print(utils.find_missing(times = btc['time']).collect())

shape: (1_440, 2)
┌─────────────────────┬───────┐
│ Times               ┆ diff  │
│ ---                 ┆ ---   │
│ datetime[ms]        ┆ bool  │
╞═════════════════════╪═══════╡
│ 2023-02-13 00:00:00 ┆ false │
│ 2023-02-13 00:01:00 ┆ false │
│ 2023-02-13 00:02:00 ┆ false │
│ 2023-02-13 00:03:00 ┆ false │
│ …                   ┆ …     │
│ 2023-02-13 23:56:00 ┆ false │
│ 2023-02-13 23:57:00 ┆ false │
│ 2023-02-13 23:58:00 ┆ false │
│ 2023-02-13 23:59:00 ┆ false │
└─────────────────────┴───────┘


In [5]:
df = btc.to_pandas()

day1 = btc.with_columns(
    pl.from_epoch('time', time_unit = 'ms')
).filter(
    pl.col('time').cast(pl.Date).is_between(
        pl.date(year = 2023, month = 2, day = 1),
        pl.date(year = 2023, month = 2, day = 3)
    )
).to_pandas()

print(day1)


                    time     open     high      low    close   volume
0    2023-02-01 00:00:00  23119.5  23134.2  23119.4  23129.9  449.836
1    2023-02-01 00:01:00  23130.0  23131.2  23117.4  23117.6  286.361
2    2023-02-01 00:02:00  23117.6  23135.8  23117.6  23135.6  213.671
3    2023-02-01 00:03:00  23135.5  23161.4  23135.5  23154.1  628.889
4    2023-02-01 00:04:00  23154.0  23154.0  23126.6  23126.8  246.057
...                  ...      ...      ...      ...      ...      ...
4315 2023-02-03 23:55:00  23421.6  23445.9  23420.8  23443.7  357.050
4316 2023-02-03 23:56:00  23443.8  23443.8  23431.4  23432.5  106.692
4317 2023-02-03 23:57:00  23432.5  23433.1  23428.1  23428.8   41.993
4318 2023-02-03 23:58:00  23428.8  23431.2  23426.5  23426.5   46.988
4319 2023-02-03 23:59:00  23426.6  23429.0  23424.7  23424.7   56.312

[4320 rows x 6 columns]


In [6]:
utils.FinancePlot(data = day1, log = True)

In [7]:
utils.check_stationarity(np.log10(df.close))

ADF Statistic: -1.600162
p-value: 0.483512
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Non-stationary


In [8]:
#Differentiated Time Series
close_diff = df.close.diff()[1:]

utils.check_stationarity(close_diff)

ADF Statistic: -42.162019
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Stationary


In [300]:
close_log_returns = np.log(df.close) - np.log(df.close.shift(1))
close_log_returns = close_log_returns[1:]

#utils.check_stationarity(close_log_returns)

close_log_returns1 = np.log(day1.close) - np.log(day1.close.shift(1))
close_log_returns1 = close_log_returns1[1:]

utils.check_stationarity(close_log_returns1)

ADF Statistic: -12.369307
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
Stationary


In [426]:
utils.plot_ACF(data = close_log_returns1, lags = 30)

In [427]:
train, test = close_log_returns[:len(close_log_returns) - 19], close_log_returns[-20:] 

In [419]:
from statsforecast.models import AutoARIMA

In [420]:
model = AutoARIMA(
    start_p = 1,
    start_q = 1,
    max_p = 5,
    max_q = 5,
    D = 1,
    d = 1,
    seasonal = True,
    season_length = 7,
    start_P = 0,
    start_Q = 0
).fit(np.array(close_log_returns1))

In [423]:
prediction100 = model.predict(h = 100, level = [80])

In [424]:
utils.plot_predictions(
    test = close_log_returns1[-100:], 
    prediction = prediction100['mean'], 
    bounds = [list(prediction100['hi-80']), list(prediction100['lo-80'])]
)

In [305]:
from statsforecast.models import MSTL, AutoETS
from statsforecast.utils import ConformalIntervals

In [312]:
mstl = MSTL(
    season_length = [2,7,10,12,15],
    trend_forecaster = AutoARIMA(
        prediction_intervals = ConformalIntervals(h = 8, n_windows = 2)
    )
)

In [327]:
print(close_log_returns1[:len(close_log_returns1) - 8])

test = close_log_returns1[-8:]

print(test)

1      -0.000532
2       0.000778
3       0.000799
4      -0.001180
5      -0.001922
          ...   
4307    0.000290
4308   -0.000055
4309   -0.000256
4310    0.000243
4311   -0.000081
Name: close, Length: 4311, dtype: float64
4312   -0.000841
4313   -0.000265
4314    0.000235
4315    0.000939
4316   -0.000478
4317   -0.000158
4318   -0.000098
4319   -0.000077
Name: close, dtype: float64


In [322]:
predictionMSTL = mstl.forecast(y = np.array(close_log_returns1[:len(close_log_returns1) - 8]), h = 8, level = [80])

In [323]:
print(predictionMSTL)

{'mean': array([ 2.05062274e-04, -1.30946424e-04,  1.57525101e-05, -1.06908105e-04,
        3.78703758e-04, -9.39476120e-05,  1.47282771e-04, -4.52746005e-04]), 'lo-80': array([ 7.33018729e-06, -5.61188876e-04, -1.58862565e-04, -5.51859313e-04,
        3.04801972e-04, -2.18037217e-04, -2.01637196e-04, -5.93094575e-04]), 'hi-80': array([ 4.02794361e-04,  2.99296029e-04,  1.90367586e-04,  3.38043103e-04,
        4.52605544e-04,  3.01419933e-05,  4.96202737e-04, -3.12397435e-04])}


In [324]:
utils.plot_predictions(
    test = test, 
    prediction = predictionMSTL['mean'], 
    bounds = [list(predictionMSTL['hi-80']), list(predictionMSTL['lo-80'])]
)

In [470]:
import sktime.datasets as data

'''
MiniRocket takes the time series and does a kernal computation to compute a kernal representation of the time series to featurize.
'''

from sktime.transformations.panel.rocket import MiniRocket

import pandas as pd

In [507]:
'''
Description:
    20 Batches of arrays of values with length 23. Labeled with a 1 or a 2.
'''

X_train, y_train = data.load_unit_test(split="train")
X_test, y_test = data.load_unit_test(split="test") 

In [487]:
X_train = MiniRocket(num_kernels = 128).fit_transform(X_train)

In [488]:
X_train = pd.DataFrame({"dim_0": pd.Series([row[1] for row in X_train.iterrows()])})

In [489]:
from sktime.classification.dictionary_based import BOSSEnsemble

In [495]:
clf = BOSSEnsemble(max_ensemble_size=3).fit(X_train, y_train)

In [496]:
y_pred = clf.predict(X_test)

In [497]:
score = clf.score(X_test, y_test)

In [498]:
print(score)

0.8636363636363636


In [506]:
from sktime.classification.kernel_based import Arsenal


In [512]:
ars = Arsenal(num_kernels = 10000, n_estimators = 5).fit(X_train, y_train)

In [513]:
ars.score(X_test, y_test)

0.9090909090909091